# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
#Read output csv from part 1
city_csv = "../../output_data/cities.csv"

city_df = pd.read_csv(city_csv)

city_df

,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Casablanca,33.5928,-7.6192,61.02,63,21,11.01,MA,1650406879
1,1,Olovyannaya,50.9500,115.5667,39.40,46,100,16.55,RU,1650406879
2,2,Lavrentiya,65.5833,-171.0000,29.07,74,3,6.15,RU,1650406880
3,3,Ubinskoye,55.3068,79.6802,36.48,63,100,6.80,RU,1650406880
4,4,New Norfolk,-42.7826,147.0587,56.70,76,97,3.11,AU,1650406881
...,...,...,...,...,...,...,...,...,...,...
498,498,Chekhov,55.1431,37.4711,44.85,95,89,10.96,RU,1650407635
499,499,Peterhead,57.5058,-1.7981,43.66,78,79,7.27,GB,1650407635
500,500,Sinaloa,25.0000,-107.5000,90.10,33,0,5.99,MX,1650407385
501,501,Ulundi,-28.3352,31.4162,54.63,95,1,3.65,ZA,1650407635


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#store lat and lng as locations
locations = city_df[["Lat", "Lon"]].astype(float)

humidity = city_df["Humidity"].astype(float)

#create heatmap layer
fig = gmaps.figure()

#create figure specs
figure_layout = {
    'width': '1000px',
    'height': '500px',
    'border': '1 px solid black',
    'padding': '1 px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center = (10,0), zoom_level=2)

#format heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1 px solid black', height='500px', margin='0 auto 0 auto', padding='1 px', …

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
#Calculate unit conversions from celcius and kmh to farenheit and mph
city_df['Max Temp'] = ((9/5) * city_df['Max Temp']) + 32
city_df['Wind Speed'] = (city_df['Wind Speed']) * 2.23694

#Drop all data outside of temperature range of 70-80 Farenheit
perfect_temp = city_df.drop(city_df[city_df['Max Temp'] <= 70].index, inplace=True)
perfect_temp = city_df.drop(city_df[city_df['Max Temp'] >= 80].index, inplace=False)

# filter dataframe to include windspeeds less than 10 mph
perfect_temp_wind = perfect_temp.drop(perfect_temp[perfect_temp['Wind Speed'] >= 10].index, inplace=False)

#Filter datatframe to include data with 0% cloudiness
perfect_temp_wind_clouds = perfect_temp_wind.drop(perfect_temp_wind[perfect_temp_wind['Cloudiness'] > 0].index, inplace=False)

#Create new column named hotel name
perfect_temp_wind_clouds["Hotel Name"] = ""

#Change name and create dataframe
hotel_df = perfect_temp_wind_clouds

hotel_df = hotel_df.reset_index(drop=True)

hotel_df = pd.DataFrame(hotel_df)

hotel_df

,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name


In [15]:
#base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Create search variables
parameters = {
    "type": 'hotel',
    "radius": 5000,
    "keyword": 'lodging',
    "key": g_key,
}

#Iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lon = row['Lon']
    
    parameters['location'] = f'{lat}, {lon}'
    
#run request using parameters dictionary
response = requests.get(base_url, parameters=parameters).json()

results = response['results']

#Modify dataframe with hotel name results
try:
        
    hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
except: (KeyError, IndexError):
            
     print('Missing field/result....skipping.')

SyntaxError: invalid syntax (Temp/ipykernel_7448/3475147767.py, line 29)

In [5]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

KeyError: "['Lng'] not in index"

In [ ]:
# Add marker layer ontop of heat map


# Display figure
